# Keras 실습

#### Tutorial on CNN implementation for own data set in keras(Anuj shah)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os,cv2
import time

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam

In [7]:
#PATH = os.getcwd() #C:\\Users\\v\\
# Define data path
#data_path = PATH + '/own_data_cnn_implementation_keras.git/trunk/data'
data_path = 'C:/Users/v/OneDrive/own_data_cnn_implementation_keras-master/data'

data_dir_list = os.listdir(data_path) #data안의 list 들을 읽는 듯 (4개)

print(data_dir_list) 


#for dataset in data_dir_list:
#    try:
#        img_list=os.listdir(data_path+'/'+dataset)
#        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
#    except NotADirectoryError:
#        pass

['cats', 'dogs', 'horses', 'Humans']


In [8]:
img_rows=128
img_cols=128
num_channel=1 # 흑백
num_epoch=20

# Define the number of classes
num_classes = 4

img_data_list=[]

In [9]:
for dataset in data_dir_list: #리스트의 원소가 문자열 이라도 for문 사용이 가능 하다 
    img_list=os.listdir(data_path+'/'+dataset) #낱개 그림 파일들
    #print(img_list)
    print ('Loaded the images of dataset '+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY) #파이썬은 BGR
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize) #202 * 4개  
        
print(input_img)

Loaded the images of dataset cats

Loaded the images of dataset dogs

Loaded the images of dataset horses

Loaded the images of dataset Humans

[[194 192 198 ..., 139 138 137]
 [194 192 198 ..., 140 139 138]
 [195 192 198 ..., 142 140 140]
 ..., 
 [220 220 221 ..., 171 171 171]
 [220 220 221 ..., 172 172 172]
 [220 220 221 ..., 173 173 173]]


In [14]:
img_data = np.array(img_data_list) #리스트를 배열 형태로 바꾼다.(?)
print(img_data)
img_data = img_data.astype('float32') #형 변환 연산자
#img_data /= 255 # normalization


print (img_data.shape) 

[[[ 43  44  43 ..., 164 203 193]
  [ 42  41  41 ..., 159 190 195]
  [ 40  36  35 ..., 159 183 193]
  ..., 
  [ 21  18  18 ...,  43  74  28]
  [ 27  21  21 ...,  39  37  35]
  [ 29  20  20 ...,  52  40  37]]

 [[ 35  16  32 ..., 153 152 148]
  [ 45  45  48 ..., 153 154 149]
  [ 52  57  55 ..., 154 156 151]
  ..., 
  [168 159 150 ..., 169 171 155]
  [163 156 156 ..., 171 169 150]
  [144 134 137 ..., 171 168 151]]

 [[222 223 223 ..., 245 235 247]
  [219 221 223 ..., 240 237 237]
  [224 223 220 ..., 243 243 242]
  ..., 
  [205 207 204 ..., 212 212 213]
  [213 210 206 ..., 212 212 213]
  [217 210 200 ..., 213 212 213]]

 ..., 
 [[224 225 226 ..., 211 210 210]
  [220 220 221 ..., 207 207 206]
  [214 214 215 ..., 202 202 202]
  ..., 
  [ 51  70  54 ...,  56  86  63]
  [ 56  67  46 ...,  61  76  60]
  [ 49  42  79 ..., 100  69  64]]

 [[ 20  17  20 ...,  40  41  48]
  [ 24  15  33 ...,  59  61  43]
  [ 35  14  17 ...,  27  30  38]
  ..., 
  [ 98  99  91 ..., 108 120  87]
  [ 87  97 116 ..., 1

#### Keras와 Tensorflow의 차이는 Image dimension ordering

In [6]:
if num_channel==1: #흑백
	if K.image_dim_ordering()=='th': #theano
		img_data= np.expand_dims(img_data, axis=1) #1차원임을 표시
		print (img_data.shape)
	else:
		img_data= np.expand_dims(img_data, axis=4) 
		print (img_data.shape)
		
else:
	if K.image_dim_ordering()=='th':
		img_data=np.rollaxis(img_data,3,1)
		print (img_data.shape)
		

(808, 1, 128, 128)


In [14]:
'''
USE_SKLEARN_PREPROCESSING=False

if USE_SKLEARN_PREPROCESSING:
	# using sklearn for preprocessing
	from sklearn import preprocessing
	
	def image_to_feature_vector(image, size=(128, 128)):
		# resize the image to a fixed size, then flatten the image into
		# a list of raw pixel intensities
		return cv2.resize(image, size).flatten()
	
	img_data_list=[]
	for dataset in data_dir_list:
		img_list=os.listdir(data_path+'/'+ dataset)
		print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
		for img in img_list:
			input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
			input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
			input_img_flatten=image_to_feature_vector(input_img,(128,128))
			img_data_list.append(input_img_flatten)
	
	img_data = np.array(img_data_list)
	img_data = img_data.astype('float32')
	print (img_data.shape)
    
    #zero mean and unit variance of data(flatten the data)
	img_data_scaled = preprocessing.scale(img_data)
	print (img_data_scaled.shape) #(808, 16384)
	
	print (np.mean(img_data_scaled))
	print (np.std(img_data_scaled))
	
	print (img_data_scaled.mean(axis=0))
	print (img_data_scaled.std(axis=0))
	
	if K.image_dim_ordering()=='th':
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],num_channel,img_rows,img_cols)
		print (img_data_scaled.shape)
		
	else:
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],img_rows,img_cols,num_channel)
		print (img_data_scaled.shape)
	
	
	if K.image_dim_ordering()=='th':
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],num_channel,img_rows,img_cols)
		print (img_data_scaled.shape)
		
	else:
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],img_rows,img_cols,num_channel)
		print (img_data_scaled.shape)

if USE_SKLEARN_PREPROCESSING:
	img_data=img_data_scaled
'''

"\nUSE_SKLEARN_PREPROCESSING=False\n\nif USE_SKLEARN_PREPROCESSING:\n\t# using sklearn for preprocessing\n\tfrom sklearn import preprocessing\n\t\n\tdef image_to_feature_vector(image, size=(128, 128)):\n\t\t# resize the image to a fixed size, then flatten the image into\n\t\t# a list of raw pixel intensities\n\t\treturn cv2.resize(image, size).flatten()\n\t\n\timg_data_list=[]\n\tfor dataset in data_dir_list:\n\t\timg_list=os.listdir(data_path+'/'+ dataset)\n\t\tprint ('Loaded the images of dataset-'+'{}\n'.format(dataset))\n\t\tfor img in img_list:\n\t\t\tinput_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )\n\t\t\tinput_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)\n\t\t\tinput_img_flatten=image_to_feature_vector(input_img,(128,128))\n\t\t\timg_data_list.append(input_img_flatten)\n\t\n\timg_data = np.array(img_data_list)\n\timg_data = img_data.astype('float32')\n\tprint (img_data.shape)\n    \n    #zero mean and unit variance of data(flatten the data)\n\timg_data_scaled = prep

In [7]:
# Define the number of classes
num_classes = 4

print (img_data.shape)

num_of_samples = img_data.shape[0] # 808
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:202]=0
labels[202:404]=1
labels[404:606]=2
labels[606:]=3

names = ['cats','dogs','horses','humans']

In [8]:
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes) #★★★
print(Y) # 808개 [1,0,0,0] ~ [0,0,0,1]
#[0,0,0,0] ~ [0,0,0,3] 이 될줄 알았는데 그건 아니었다.

[[ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


In [9]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2) #808개의 순서를 Shuffle

# Split the dataset
#1/5로 Split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [10]:
# Defining the model
input_shape=img_data[0].shape
					
model = Sequential()

model.add(Conv2D(32, 3,3,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

c:\users\v\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(1, 128, 1..., padding="same")`
  
c:\users\v\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
c:\users\v\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  del sys.path[0]


In [13]:
# Viewing model_configuration
model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 128, 128)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 128, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 126, 126)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 126, 126)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 63, 63)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 63, 63)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 61, 61)        18496     
__________

True

In [12]:
# Training
T1 = time.time()
hist = model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))
T2 = time.time()
print(T2-T1)

Train on 646 samples, validate on 162 samples
Epoch 1/20
646/646 [==============================] - 146s - loss: 1.6668 - acc: 0.2848 - val_loss: 1.4120 - val_acc: 0.2963
Epoch 2/20
646/646 [==============================] - 157s - loss: 1.3570 - acc: 0.3607 - val_loss: 1.3416 - val_acc: 0.4383
Epoch 3/20
646/646 [==============================] - 154s - loss: 1.2158 - acc: 0.4628 - val_loss: 1.2798 - val_acc: 0.4321
Epoch 4/20
646/646 [==============================] - 157s - loss: 1.0534 - acc: 0.5480 - val_loss: 1.2399 - val_acc: 0.4568
Epoch 5/20
646/646 [==============================] - 139s - loss: 0.9416 - acc: 0.6146 - val_loss: 1.1616 - val_acc: 0.4630
Epoch 6/20
646/646 [==============================] - 163s - loss: 0.7854 - acc: 0.6842 - val_loss: 1.0920 - val_acc: 0.5432
Epoch 7/20
646/646 [==============================] - 129s - loss: 0.6647 - acc: 0.7399 - val_loss: 1.2357 - val_acc: 0.4877
Epoch 8/20
646/646 [==============================] - 143s - loss: 0.5736 - acc

In [14]:
# Training with callbacks
from keras import callbacks

filename='model_train_new.csv'
csv_log=callbacks.CSVLogger(filename, separator=',', append=False)

early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

filepath="Best-weights-my_model-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"

# 가장 작은 것을 저장.
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [csv_log,early_stopping,checkpoint]

hist = model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test),callbacks=callbacks_list)


Train on 646 samples, validate on 162 samples
Epoch 1/20
640/646 [============================>.] - ETA: 1s - loss: 0.0920 - acc: 0.9703Epoch 00000: val_loss improved from inf to 2.67929, saving model to Best-weights-my_model-000-0.0911-0.9706.hdf5


ImportError: `save_model` requires h5py.

#### Early stopping : Stop training when a monitores quantity has stop improving

In [ ]:
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(num_epoch)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Evaluating the model

#score = model.evaluate(X_test, y_test, show_accuracy=True, verbose=0)
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

test_image = X_test[0:1]
print (test_image.shape)

print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

In [ ]:
# Testing a new image
test_image = cv2.imread('data/Humans/rider-8.jpg')
test_image=cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_image=cv2.resize(test_image,(128,128))
test_image = np.array(test_image)
test_image = test_image.astype('float32')
test_image /= 255
print (test_image.shape)
   
if num_channel==1:
	if K.image_dim_ordering()=='th':
		test_image= np.expand_dims(test_image, axis=0)
		test_image= np.expand_dims(test_image, axis=0)
		print (test_image.shape)
	else:
		test_image= np.expand_dims(test_image, axis=3) 
		test_image= np.expand_dims(test_image, axis=0)
		print (test_image.shape)
		
else:
	if K.image_dim_ordering()=='th':
		test_image=np.rollaxis(test_image,2,0)
		test_image= np.expand_dims(test_image, axis=0)
		print (test_image.shape)
	else:
		test_image= np.expand_dims(test_image, axis=0)
		print (test_image.shape)
		
        
# Predicting the test image
print((model.predict(test_image)))
print(model.predict_classes(test_image))

In [ ]:
# Visualizing the intermediate layer
def get_featuremaps(model, layer_idx, X_batch):
	get_activations = K.function([model.layers[0].input, K.learning_phase()],[model.layers[layer_idx].output,])
	activations = get_activations([X_batch,0])
	return activations

layer_num=3
filter_num=0

activations = get_featuremaps(model, int(layer_num),test_image)

print (np.shape(activations)) #(1, 1, 32, 128, 128) <-theano
feature_maps = activations[0][0]      
print (np.shape(feature_maps))

if K.image_dim_ordering()=='th':
	feature_maps=np.rollaxis((np.rollaxis(feature_maps,2,0)),2,0)
print (feature_maps.shape)

fig=plt.figure(figsize=(16,16))
plt.imshow(feature_maps[:,:,filter_num],cmap='gray')
plt.savefig("featuremaps-layer-{}".format(layer_num) + "-filternum-{}".format(filter_num)+'.jpg')

num_of_featuremaps=feature_maps.shape[2]
fig=plt.figure(figsize=(16,16))	
plt.title("featuremaps-layer-{}".format(layer_num))
subplot_num=int(np.ceil(np.sqrt(num_of_featuremaps)))
for i in range(int(num_of_featuremaps)):
	ax = fig.add_subplot(subplot_num, subplot_num, i+1)
	#ax.imshow(output_image[0,:,:,i],interpolation='nearest' ) #to see the first filter
	ax.imshow(feature_maps[:,:,i],cmap='gray')
	plt.xticks([])
	plt.yticks([])
	plt.tight_layout()
plt.show()
fig.savefig("featuremaps-layer-{}".format(layer_num) + '.jpg')

In [ ]:
# Printing the confusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = model.predict(X_test)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
#y_pred = model.predict_classes(X_test)
#print(y_pred)
target_names = ['class 0(cats)', 'class 1(Dogs)', 'class 2(Horses)','class 3(Humans)']
					
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

In [ ]:
# Plotting the confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = (confusion_matrix(np.argmax(y_test,axis=1), y_pred))

np.set_printoptions(precision=2)

plt.figure()

# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix')
#plt.figure()
# Plot normalized confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=target_names, normalize=True,
#                      title='Normalized confusion matrix')
#plt.figure()
plt.show()

In [ ]:
# Saving and loading model and weights
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

model.save('model.hdf5')
loaded_model=load_model('model.hdf5')